Baseball Prediction: 3b - Augment DataFrame with Odds Data

In the previous notebook, we got historical odds data from oddsshark.com and saved them as a set of csv files (with a particular naming convention).

In this notebook we will load that data and augment our primary (game-level) data frame so that it includes this odds data - specifically, the implied probabilities and the over/under, for each game.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lxml
import html5lib
%matplotlib inline
from urllib.request import urlopen
import time
import structureboost as stb
import ml_insights as mli
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
df=pd.read_csv('df_bp1.csv', low_memory=False)

Plan of attack

- Create a dictionary structure to enable us to easily go to a specific game for a particular team and season.
- Iterate through the rows of our main dataframe, look up the home and visiting team for that game to get their odds. (Note: need two different lookups per game)

In [4]:
# Use the same mapping as before
# (more elegant to save to file...)

oddsshark_num_to_team_dict = {}
oddsshark_num_to_team_dict[26995]='PHI'
oddsshark_num_to_team_dict[26996]='SDN'
oddsshark_num_to_team_dict[26997]='SFN'
oddsshark_num_to_team_dict[26998]='ANA'
oddsshark_num_to_team_dict[26999]='DET'
oddsshark_num_to_team_dict[27000]='CIN'
oddsshark_num_to_team_dict[27001]='NYA'
oddsshark_num_to_team_dict[27002]='TEX'
oddsshark_num_to_team_dict[27003]='TBA'
oddsshark_num_to_team_dict[27004]='COL'
oddsshark_num_to_team_dict[27005]='MIN'
oddsshark_num_to_team_dict[27006]='KCA'
oddsshark_num_to_team_dict[27007]='ARI'
oddsshark_num_to_team_dict[27008]='BAL'
oddsshark_num_to_team_dict[27009]='ATL'
oddsshark_num_to_team_dict[27010]='TOR'
oddsshark_num_to_team_dict[27011]='SEA'
oddsshark_num_to_team_dict[27012]='MIL'
oddsshark_num_to_team_dict[27013]='PIT'
oddsshark_num_to_team_dict[27014]='NYN'
oddsshark_num_to_team_dict[27015]='LAN'
oddsshark_num_to_team_dict[27016]='OAK'
oddsshark_num_to_team_dict[27017]='WAS'
oddsshark_num_to_team_dict[27018]='CHA'
oddsshark_num_to_team_dict[27019]='SLN'
oddsshark_num_to_team_dict[27020]='CHN'
oddsshark_num_to_team_dict[27021]='BOS'
oddsshark_num_to_team_dict[27022]='MIA'
oddsshark_num_to_team_dict[27023]='HOU'
oddsshark_num_to_team_dict[27024]='CLE'
 
# Use the saved files to get the odds information
# We create a dict based on team and season for easy lookup
df_odds_dict={}
for i in range(26995, 27025):
    team_name = oddsshark_num_to_team_dict[i]
    df_odds_dict[team_name] = {}
    print(team_name)
    for season in range(2019,2023):
        fname = 'oddsshark_'+team_name+'_'+str(season)+'.csv'
        df_temp = pd.read_csv('/Users/antiprotons/Desktop/beatingVegas/oddshark/'+fname)
        df_temp['date_dblhead'] = (df_temp.date_numeric.astype(str) + df_temp.dblheader_num.astype(str)).astype(int)
        df_temp.set_index('date_dblhead', inplace=True)
        df_odds_dict[team_name][season] = df_temp

PHI
SDN
SFN
ANA
DET


CIN
NYA
TEX
TBA
COL
MIN
KCA
ARI
BAL
ATL
TOR
SEA
MIL
PIT
NYN
LAN
OAK
WAS
CHA
SLN
CHN
BOS
MIA
HOU
CLE


Augment our main dataframe

In [5]:
# Again, we iterate through the main dataframe
# get the team, season, game and then get
# the relevant info from the odds dictionary

implied_prob_h = np.zeros(df.shape[0])
implied_prob_v = np.zeros(df.shape[0])
over_under = np.zeros(df.shape[0])
ou_result = np.full(df.shape[0],'', dtype=object)
for ind, row in df.iterrows():
    if (ind%1000)==0:
        print(ind)
    if row.season<2019:
        continue
    else:
        season = row['season']
        home_team = row['team_h']
        visit_team = row['team_v']
        home_game_no = row['game_no_h']
        visit_game_no = row['game_no_v']
        date_dblh = row['date_dblhead']
        try:
            implied_prob_h[ind] = df_odds_dict[home_team][season].loc[date_dblh,'prob_implied']
            over_under[ind] = df_odds_dict[home_team][season].loc[date_dblh,'Total']
            ou_result[ind] = df_odds_dict[home_team][season].loc[date_dblh,'OU']
        except KeyError:
            print(f'Game not found wrt home_team:{home_team} vs {visit_team} date_dbl {date_dblh}')
        try:
            implied_prob_v[ind] = df_odds_dict[visit_team][season].loc[date_dblh,'prob_implied']
        except KeyError:
            print(f'Game not found wrt visit_team:{visit_team} vs {home_team} date_dbl {date_dblh}')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [6]:
df['implied_prob_h'] = implied_prob_h
df['implied_prob_v'] = implied_prob_v
df['implied_prob_h_mid'] = (implied_prob_h + (1-implied_prob_v))/2
df['over_under_line']=over_under
df['over_under_result']=ou_result

Drop games with no odds...

In [7]:
df[(df.season>=2019) & (df.implied_prob_h==0)]

date  dblheader_code day_of_week team_v league_v  game_no_v  \
201803  20210626               0         Sat    SEA       AL         78   
202086  20210721               1         Wed    SDN       NL         98   
202087  20210721               2         Wed    SDN       NL         99   

       team_h league_h  game_no_h  runs_v  runs_h  outs_total day_night  \
201803    CHA       AL         76       3       2          54         D   
202086    ATL       NL         94       3       2          42         D   
202087    ATL       NL         95       6       5          42         N   

              completion_info forfeit_info protest_info ballpark_id  \
201803  20210627,CHI12,0,0,15          NaN          NaN       CHI12   
202086                    NaN          NaN          NaN       ATL03   
202087  20210924,SAN02,5,4,27          NaN          NaN       ATL03   

        attendance  game_minutes linescore_v linescore_h  AB_v  H_v  2B_v  \
201803     30017.0         197.0   000011001   000011000  34.0  7.0   0.0   
202086     28621.0         156.0     0001200     0000020  26.0  7.0   2.0   
202087        -1.0         126.0     4100001     0400010  26.0  6.0   2.0   

        3B_v  HR_v  RBI_v  SH_v  SF_v  HBP_v  BB_v  IBB_v  SO_v  SB_v  CS_v  \
201803   0.0   3.0    3.0   0.0   0.0    0.0   3.0    0.0   9.0   0.0   0.0   
202086   0.0   1.0    3.0   0.0   1.0    0.0   4.0    0.0   6.0   0.0   1.0   
202087   0.0   1.0    5.0   0.0   1.0    0.0   6.0    2.0   7.0   1.0   0.0   

        GIDP_v  CI_v  LOB_v  P_num_v  ERind_v  ERteam_v  WP_v  balk_v  PO_v  \
201803     1.0   0.0    7.0      7.0      2.0       2.0   0.0     0.0  27.0   
202086     0.0   0.0    7.0      3.0      1.0       1.0   0.0     0.0  21.0   
202087     0.0   0.0    6.0      7.0      5.0       5.0   0.0     0.0  21.0   

        ASST_v  ERR_v  PB_v  DP_v  TP_v  AB_h  H_h  2B_h  3B_h  HR_h  RBI_h  \
201803     9.0    0.0   0.0   3.0   0.0  29.0  6.0   0.0   0.0   0.0    2.0   
202086    10.0    1.0   1.0   0.0   0.0  25.0  5.0   1.0   0.0   0.0    1.0   
202087     5.0    0.0   0.0   1.0   0.0  24.0  5.0   2.0   0.0   1.0    5.0   

        SH_h  SF_h  HBP_h  BB_h  IBB_h  SO_h  SB_h  CS_h  GIDP_h  CI_h  LOB_h  \
201803   0.0   1.0    1.0   3.0    0.0  11.0   1.0   0.0     3.0   0.0    5.0   
202086   1.0   1.0    0.0   0.0    0.0   3.0   0.0   0.0     0.0   0.0    4.0   
202087   0.0   1.0    0.0   6.0    0.0   6.0   0.0   0.0     1.0   0.0    5.0   

        P_num_h  ERind_h  ERteam_h  WP_h  balk_h  PO_h  ASST_h  ERR_h  PB_h  \
201803      3.0      3.0       3.0   0.0     0.0  27.0     7.0    1.0   0.0   
202086      3.0      3.0       3.0   2.0     0.0  21.0     8.0    0.0   0.0   
202087      4.0      6.0       6.0   2.0     0.0  21.0     7.0    0.0   0.0   

        DP_h  TP_h ump_HB_id    ump_HB_name ump_1B_id   ump_1B_name ump_2B_id  \
201803   1.0   0.0  cuzzp901     Phil Cuzzi  hallt901   Tom Hallion  rippm901   
202086   0.0   0.0  timmt901    Tim Timmons  riggj901  Jeremy Riggs  mahrn901   
202087   0.0   0.0  torrc901  Carlos Torres  mahrn901  Nick Mahrley  marqa901   

            ump_2B_name ump_3B_id      ump_3B_name ump_LF_id ump_LF_name  \
201803   Mark Ripperger  blasc901      Cory Blaser       NaN      (none)   
202086     Nick Mahrley  marqa901  Alfonso Marquez       NaN      (none)   
202087  Alfonso Marquez  riggj901     Jeremy Riggs       NaN      (none)   

       ump_RF_id ump_RF_name  mgr_id_v     mgr_name_v  mgr_id_h  \
201803       NaN      (none)  servs002  Scott Servais  larut101   
202086       NaN      (none)  tingj801  Jayce Tingler  snitb801   
202087       NaN      (none)  tingj801  Jayce Tingler  snitb801   

           mgr_name_h pitcher_id_w pitcher_name_w pitcher_id_l pitcher_name_l  \
201803   Tony LaRussa     sewap001    Paul Sewald     hendl001  Liam Hendriks   
202086  Brian Snitker     paddc001  Chris Paddack     mullk001    Kyle Muller   
202087  Brian Snitker     hudsd001  Daniel Hudson     smitw002     Will Smith   


In [8]:
indicies_to_drop = df[(df.season>=2019) & (df.implied_prob_h==0)].index
indicies_to_drop

Int64Index([201803, 202086, 202087], dtype='int64')

In [9]:
df.shape

(205524, 199)

In [10]:
df.drop(indicies_to_drop, inplace=True)
df.shape

(205521, 199)

In [11]:
df.reset_index(inplace=True, drop=True)
df.shape

(205521, 199)

In [12]:
df.to_csv('df_bp3.csv', index=False)